In [8]:
from __future__ import print_function

import os.path

import keras
from keras import applications, metrics, layers, models, regularizers, optimizers
from keras.applications import ResNet50, Xception, InceptionResNetV2
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.preprocessing.image import ImageDataGenerator

# Globals
BATCH_SIZE = 32   # tweak to your GPUs capacity
IMG_HEIGHT = 299   # ResNetInceptionv2 & Xception like 299, ResNet50 & VGG like 224
IMG_WIDTH = IMG_HEIGHT
CHANNELS = 3
DIMS = (IMG_HEIGHT,IMG_WIDTH,CHANNELS) # what an ugly holdover from a framework not even supported by it's authors
BEST_MODEL = 'keras.best.h5'

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.1,)

train_generator = train_datagen.flow_from_directory(
    'D:\ML\Dog_Breed\stanford\Images',  # this is the target directory
    target_size=(IMG_HEIGHT,IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Found 20580 images belonging to 120 classes.


In [12]:

classes = len(train_generator.class_indices)
callbacks = [ModelCheckpoint(filepath=BEST_MODEL, verbose=0, save_best_only=True),
             EarlyStopping(monitor='val_acc', patience=3, verbose=0)]

# base_model = Xception(input_shape=DIMS, weights='imagenet', include_top=False) #~
base_model = ResNet50(weights='imagenet', include_top=False, input_shape= (299,299 , 3))
#base_model = InceptionResNetV2(input_shape=DIMS, weights='imagenet', include_top=False)
for layer in base_model.layers:
    layer.trainable = False

x = base_model.output

# x = Flatten()(x)
# x = Dense(classes, activation='softmax', name='predictions')(x)

x = GlobalAveragePooling2D(name='avg_pool')(base_model.output)
x = Dense(classes, activation='softmax', name='predictions')(x)

model = Model(inputs=base_model.input, outputs=x)

In [13]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizers.Adam(1e-3),
    metrics=['acc'])

model_out = model.fit_generator(
    train_generator,
    steps_per_epoch=20580 /32,
    epochs=20,
    verbose=1,
    callbacks=callbacks)

Epoch 1/20
644/643 [==============================] - 887s 1s/step - loss: 2.4897 - acc: 0.3987


c:\users\varshith\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\keras\callbacks.py:434: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
c:\users\varshith\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\keras\callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


Epoch 2/20
644/643 [==============================] - 408s 633ms/step - loss: 1.3988 - acc: 0.6087
Epoch 3/20
644/643 [==============================] - 409s 635ms/step - loss: 1.2002 - acc: 0.6532
Epoch 4/20
644/643 [==============================] - 410s 637ms/step - loss: 1.0623 - acc: 0.6904
Epoch 5/20
644/643 [==============================] - 408s 634ms/step - loss: 0.9839 - acc: 0.7076
Epoch 6/20
644/643 [==============================] - 411s 639ms/step - loss: 0.9236 - acc: 0.7245
Epoch 7/20
644/643 [==============================] - 409s 634ms/step - loss: 0.8741 - acc: 0.7365
Epoch 8/20
644/643 [==============================] - 479s 743ms/step - loss: 0.8067 - acc: 0.7530
Epoch 9/20
644/643 [==============================] - 743s 1s/step - loss: 0.7843 - acc: 0.7614
Epoch 10/20
644/643 [==============================] - 411s 638ms/step - loss: 0.7436 - acc: 0.7710
Epoch 11/20
644/643 [==============================] - 411s 638ms/step - loss: 0.7315 - acc: 0.7739
Epoch 12/20

In [14]:
import gc
for i in range(10):
    gc.collect()

In [15]:
#model.load_weights(BEST_MODEL)
model.compile(
    optimizer=optimizers.Adam(lr=1e-4,),
    loss='categorical_crossentropy',
    metrics=['acc'])

model_out = model.fit_generator(
    train_generator,
    steps_per_epoch=20580/32,
    epochs=10,
    verbose=1,
    callbacks=callbacks)

Epoch 1/10
644/643 [==============================] - 412s 640ms/step - loss: 0.3830 - acc: 0.8813
Epoch 2/10
644/643 [==============================] - 411s 638ms/step - loss: 0.3675 - acc: 0.8860
Epoch 3/10
644/643 [==============================] - 415s 644ms/step - loss: 0.3497 - acc: 0.8923
Epoch 4/10
644/643 [==============================] - 411s 638ms/step - loss: 0.3562 - acc: 0.8912
Epoch 5/10
644/643 [==============================] - 416s 647ms/step - loss: 0.3468 - acc: 0.8927
Epoch 6/10
644/643 [==============================] - 412s 640ms/step - loss: 0.3420 - acc: 0.8946
Epoch 7/10
644/643 [==============================] - 416s 646ms/step - loss: 0.3412 - acc: 0.8931
Epoch 8/10
644/643 [==============================] - 410s 637ms/step - loss: 0.3410 - acc: 0.8959
Epoch 9/10
644/643 [==============================] - 415s 645ms/step - loss: 0.3342 - acc: 0.9005
Epoch 10/10
644/643 [==============================] - 411s 639ms/step - loss: 0.3322 - acc: 0.8999


In [16]:
model.save('model.h5')

In [17]:
import gc
for i in range(10):
    gc.collect()

In [18]:
#model.load_weights(BEST_MODEL)
# print(model.summary())
# for i, layer in enumerate(model.layers):
#     print(i, layer.name)
# See model information for last convolution layer. Xception is 126.
for layer in model.layers[:126]:
    layer.trainable = False
for layer in model.layers[126:]:
    layer.trainable = True

In [19]:
import gc
for i in range(10):
    gc.collect()

In [20]:
model.compile(
    optimizer=optimizers.Adam(lr=1e-4,),
    loss='categorical_crossentropy',
    metrics=['acc'])

model_out = model.fit_generator(
    train_generator,
    steps_per_epoch=20580/32,
    epochs=10,
    verbose=1,
    callbacks=callbacks)

Epoch 1/10
644/643 [==============================] - 1052s 2s/step - loss: 0.5517 - acc: 0.8270
Epoch 2/10
644/643 [==============================] - 982s 2s/step - loss: 0.3690 - acc: 0.8806
Epoch 3/10
644/643 [==============================] - 949s 1s/step - loss: 0.2850 - acc: 0.9100
Epoch 4/10
644/643 [==============================] - 955s 1s/step - loss: 0.2501 - acc: 0.9200
Epoch 5/10
644/643 [==============================] - 973s 2s/step - loss: 0.2132 - acc: 0.9337
Epoch 6/10
644/643 [==============================] - 1018s 2s/step - loss: 0.1834 - acc: 0.9435
Epoch 7/10
644/643 [==============================] - 1028s 2s/step - loss: 0.1692 - acc: 0.9483
Epoch 8/10
644/643 [==============================] - 991s 2s/step - loss: 0.1574 - acc: 0.9518
Epoch 9/10
644/643 [==============================] - 1011s 2s/step - loss: 0.1397 - acc: 0.9580
Epoch 10/10
644/643 [==============================] - 917s 1s/step - loss: 0.1360 - acc: 0.9593


In [21]:
model.save('model2.h5')

In [2]:
import gc
for i in range(10):
    gc.collect()

In [3]:
from keras.preprocessing import image
import numpy as np

def load_test_image(fpath):
    img = image.load_img(fpath, target_size=(IMG_WIDTH, IMG_HEIGHT))
    x = image.img_to_array(img)
    return x

Using TensorFlow backend.


In [4]:
test_labels = np.loadtxt('sample_submission.csv', delimiter=',', dtype=str, skiprows=1)
test_images = []
test_names = test_labels[:,0]

In [5]:
import gc
for i in range(3):
    gc.collect()

In [9]:
import os
for test_name in test_names:
    fname = '{}.jpg'.format(test_name)
    data = load_test_image(os.path.join('test/', fname))
    test_images.append(data)

In [ ]:
import gc
for i in range(10):
    gc.collect()

In [45]:
# from tqdm import tqdm
# for i in range(len(test_images)):
#     test_images[i] = np.asarray(test_images[i])

In [70]:
sample1 = test_images[:5000]

In [71]:
sample1 = np.asarray(sample1)

In [1]:
sample2 = test_images[5000:7000]

NameError: name 'test_images' is not defined

In [ ]:
# sample2 = np.asarray(sample2)

In [ ]:
# sample3 = test_images[7000:9000]

In [ ]:
# sample3 = np.asarray(sample3)

In [ ]:
# sample4 = test_images[9000:]
# sample4 = np.asarray(sample4)

In [ ]:
# sample = np.concatenate((sample1, sample2, sample3,sample4), axis=0)

In [11]:
test_images = np.asarray(test_images)
test_images = test_images.astype('float32')
test_images /= 255
print(test_images.shape)

(10357, 299, 299, 3)


In [34]:
from keras.models import load_model
model = load_model('keras.model.best.h5')

c:\users\varshith\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\keras\engine\saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [35]:
predictions = model.predict(test_images, verbose=1)


10357/10357 [==============================] - 924s 89ms/step


In [36]:
predictions

array([[3.5224671e-06, 6.8700438e-06, 3.2194396e-06, ..., 2.7459964e-06,
        1.5228462e-05, 4.2610150e-06],
       [1.5749101e-05, 1.0604356e-05, 7.0757628e-06, ..., 4.6747782e-06,
        1.1199888e-05, 2.7494762e-06],
       [9.2150758e-06, 2.0320600e-05, 5.1662148e-05, ..., 2.4008752e-05,
        1.7736387e-05, 1.1798283e-05],
       ...,
       [3.0064952e-05, 2.2755114e-05, 4.4923645e-04, ..., 2.8604798e-05,
        6.8865906e-05, 1.1390819e-05],
       [1.5763287e-04, 1.8952878e-05, 4.5077004e-06, ..., 3.6349641e-06,
        2.4929441e-05, 2.1077642e-04],
       [1.7854078e-04, 7.7142089e-04, 2.4581875e-04, ..., 1.5779394e-03,
        2.5095296e-04, 1.6386441e-05]], dtype=float32)

In [37]:
 test_labels = pd.read_csv('sample_submission.csv')

In [39]:
submission = pd.DataFrame(predictions)

In [42]:
list = test_labels.columns

In [43]:
list2 = list[1:]

In [45]:
submission.columns = list2

In [46]:
submission.insert(0, 'id', test_labels['id'])

In [47]:
submission.to_csv('keras_best_model.csv', index = False)

In [16]:
import pandas as pd
class_indices = sorted([ [k,v] for k, v in train_generator.class_indices.items() ], key=lambda c : c[1])
columns = [b[0] for b in class_indices]
df = pd.DataFrame(predictions,columns=columns)
df = df.assign(id = test_names)
df.to_csv("submit.csv", index=False) #,float_format='%.4f')
print(df.head())

   n02085620-Chihuahua  n02085782-Japanese_spaniel  n02085936-Maltese_dog  \
0             0.000165                    0.000162               0.213990   
1             0.000100                    0.000136               0.183092   
2             0.000138                    0.000146               0.239388   
3             0.000076                    0.000099               0.222647   
4             0.000126                    0.000126               0.208422   

   n02086079-Pekinese  n02086240-Shih-Tzu  n02086646-Blenheim_spaniel  \
0            0.006573            0.000185                    0.000035   
1            0.007152            0.000179                    0.000033   
2            0.006816            0.000227                    0.000036   
3            0.005806            0.000187                    0.000032   
4            0.006810            0.000173                    0.000030   

   n02086910-papillon  n02087046-toy_terrier  n02087394-Rhodesian_ridgeback  \
0            0.0014

In [17]:
sub = pd.read_csv('submit.csv')

In [18]:
# list  = sub.columns

In [19]:
#print(list[110:120])

In [20]:
#columns[20:40]

In [22]:
cols2  = [x.lower() for x in cols]

In [23]:
sub.columns = cols2

In [24]:
sub.head()

,chihuahua,japanese_spaniel,maltese_dog,pekinese,shih-tzu,blenheim_spaniel,papillon,toy_terrier,rhodesian_ridgeback,afghan_hound,...,pembroke,cardigan,toy_poodle,miniature_poodle,standard_poodle,mexican_hairless,dingo,dhole,african_hunting_dog,id
0,0.000165,0.000162,0.213990,0.006573,0.000185,0.000035,0.001437,0.005213,0.005026,2.665518e-07,...,0.000017,0.005426,0.000303,0.000517,0.000503,0.000052,0.000348,0.026957,0.033166,000621fb3cbb32d8935728e48679680e
1,0.000100,0.000136,0.183092,0.007152,0.000179,0.000033,0.001461,0.007238,0.006315,1.913011e-07,...,0.000012,0.003068,0.000204,0.000322,0.000487,0.000031,0.000327,0.028660,0.060261,00102ee9d8eb90812350685311fe5890
2,0.000138,0.000146,0.239388,0.006816,0.000227,0.000036,0.001673,0.005235,0.005385,2.957188e-07,...,0.000015,0.004984,0.000323,0.000438,0.000445,0.000046,0.000391,0.022662,0.037868,0012a730dfa437f5f3613fb75efcd4ce
3,0.000076,0.000099,0.222647,0.005806,0.000187,0.000032,0.002148,0.006112,0.004803,2.290649e-07,...,0.000014,0.003130,0.000285,0.000289,0.000462,0.000028,0.000372,0.019302,0.050170,001510bc8570bbeee98c8d80c8a95ec1
4,0.000126,0.000126,0.208422,0.006810,0.000173,0.000030,0.001651,0.005600,0.005007,2.301827e-07,...,0.000015,0.004605,0.000287,0.000376,0.000456,0.000040,0.000356,0.027955,0.039821,001a5f3114548acdefa3d4da05474c2e


In [25]:
df = pd.read_csv('sample_submission.csv')

In [26]:
columns = df.columns

In [27]:
sub.head()

,chihuahua,japanese_spaniel,maltese_dog,pekinese,shih-tzu,blenheim_spaniel,papillon,toy_terrier,rhodesian_ridgeback,afghan_hound,...,pembroke,cardigan,toy_poodle,miniature_poodle,standard_poodle,mexican_hairless,dingo,dhole,african_hunting_dog,id
0,0.000165,0.000162,0.213990,0.006573,0.000185,0.000035,0.001437,0.005213,0.005026,2.665518e-07,...,0.000017,0.005426,0.000303,0.000517,0.000503,0.000052,0.000348,0.026957,0.033166,000621fb3cbb32d8935728e48679680e
1,0.000100,0.000136,0.183092,0.007152,0.000179,0.000033,0.001461,0.007238,0.006315,1.913011e-07,...,0.000012,0.003068,0.000204,0.000322,0.000487,0.000031,0.000327,0.028660,0.060261,00102ee9d8eb90812350685311fe5890
2,0.000138,0.000146,0.239388,0.006816,0.000227,0.000036,0.001673,0.005235,0.005385,2.957188e-07,...,0.000015,0.004984,0.000323,0.000438,0.000445,0.000046,0.000391,0.022662,0.037868,0012a730dfa437f5f3613fb75efcd4ce
3,0.000076,0.000099,0.222647,0.005806,0.000187,0.000032,0.002148,0.006112,0.004803,2.290649e-07,...,0.000014,0.003130,0.000285,0.000289,0.000462,0.000028,0.000372,0.019302,0.050170,001510bc8570bbeee98c8d80c8a95ec1
4,0.000126,0.000126,0.208422,0.006810,0.000173,0.000030,0.001651,0.005600,0.005007,2.301827e-07,...,0.000015,0.004605,0.000287,0.000376,0.000456,0.000040,0.000356,0.027955,0.039821,001a5f3114548acdefa3d4da05474c2e


In [28]:
sub = sub.reindex_axis(columns, axis=1)

c:\users\varshith\anaconda3\envs\tensorflow_gpuenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """Entry point for launching an IPython kernel.


In [29]:
#sub.to_csv('please_god_save_me.csv', index = False)

In [30]:
len(set(cols2) & set(columns))

121

In [1]:
#print(cols2)

In [32]:
sub.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.001172,2.665518e-07,0.033166,0.003352,0.000006,1.659058e-07,0.000204,0.000422,0.000684,...,0.000303,0.005213,0.001525,0.000013,0.000373,0.000346,0.000033,0.013465,0.000045,0.003971
1,00102ee9d8eb90812350685311fe5890,0.001156,1.913011e-07,0.060261,0.002657,0.000005,1.108198e-07,0.000150,0.000424,0.000756,...,0.000204,0.007238,0.001304,0.000010,0.000408,0.000311,0.000029,0.011310,0.000029,0.002934
2,0012a730dfa437f5f3613fb75efcd4ce,0.000850,2.957188e-07,0.037868,0.003556,0.000005,1.507995e-07,0.000201,0.000437,0.000671,...,0.000323,0.005235,0.001623,0.000012,0.000380,0.000359,0.000038,0.011882,0.000041,0.003736
3,001510bc8570bbeee98c8d80c8a95ec1,0.000979,2.290649e-07,0.050170,0.004077,0.000004,1.019537e-07,0.000153,0.000371,0.000775,...,0.000285,0.006112,0.001593,0.000010,0.000359,0.000362,0.000031,0.010338,0.000031,0.002936
4,001a5f3114548acdefa3d4da05474c2e,0.000905,2.301827e-07,0.039821,0.003181,0.000005,1.340415e-07,0.000189,0.000415,0.000724,...,0.000287,0.005600,0.001407,0.000010,0.000359,0.000354,0.000031,0.013313,0.000036,0.003496


In [33]:
sub.to_csv('should_work.csv', index = False)